In [5]:
%pip install easyocr
%pip install -U sentence-transformers
%pip install usearch pillow
%pip install matplotlib
%pip -q install faiss-cpu

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import easyocr
import cv2
import numpy as np
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from usearch.index import Index
import os
from tqdm.notebook import tqdm

/Users/quan238/personal/Build/competitions/AIC_2024/AIC/OCR_Process/.conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
def prepare_ocr_text(ocr_list: List[str]) -> str:
    """
    Chuẩn bị văn bản OCR để sử dụng với Sentence BERT.

    Args:
        ocr_list (List[str]): Danh sách các đoạn văn bản từ OCR.

    Returns:
        str: Văn bản đã được chuẩn bị.
    """
    filtered_list = [item.strip() for item in ocr_list if item.strip()]

    text = ". ".join(filtered_list)

    if not text.endswith("."):
        text += "."

    return text


def perform_ocr(image_path: str) -> str:
    """
    Thực hiện OCR trên 1 hình ảnh và trả về văn bản được trích xuất
    Args:
        image_path (str): Đường dẫn đến file hình ảnh.

    Returns:
        str: Văn bản được trích xuất từ hình ảnh.
    """
    result = reader.readtext(
        image_path
    )  # thực hiện đọc, và trả về 1 tuple (bounding box, đoạn text scan được, điểm confidence [0, 1])
    # Bounding box ko quan trọng
    # đoạn text scan được: List[str]: vd ["hôm nay ăn cơm", "TPHCM: Biến cố tai nạn xe", ...]
    # confidence score: Líst[float]: [0.95, 0.1,...]
    # result: chủ yếu là list các chuỗi -> prepare_ocr_text nối lại biến nó thành 1 câu duy nhất
    sentence = prepare_ocr_text([text for _, text, _ in result])
    print(sentence)
    return sentence  # trả về 1 câu tiếng Việt


def get_embedding(text: str) -> np.ndarray:
    """
    Tạo vector embedding cho một đoạn văn bản sử dụng Vietnamese Sentence BERT.

    Args:
        text (str): Đoạn văn bản cần tạo embedding.

    Returns:
        np.ndarray: Vector embedding của đoạtqvăn bản.
    """
    return model.encode(text)  # Encode 1 câu text -> vector dimension 768


def process_image_and_build_index(image_folder: str) -> Index:
    """
    Xử lý tất cả hình ảnh trong một thư mục, thực hiện OCR, tạo embedding và xây dựng index USearch.

    Args:
        image_folder (str): Đường dẫn đến thư mục chứa hình ảnh.

    Returns:
        Index: Đối tượng Index của USearch đã được xây dựng.
    """
    global global_index2imgpath
    index = Index(
        ndim=768, metric="cosine"
    )  # 768 là kích thước của SBERT embeddings, cái này là khởi tạo Usearch
    # Usearch sẽ có một số API chính
    # usearch.index (ở đây là index trong func này): Khởi tạo Usearch.index
    # usearch.index.add(i, vector): thêm vector vào usearch, na ná như nhét vector vào list, và nhét (thứ tự, vector) theo tuple
    # usearch.search(vector, k:int): vector là từ thằng query, sẽ nói thêm ở function search, còn k là số lượng vector trả về mà nó na ná như query
    image_paths = [
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.endswith((".jpg", ".png", ".webp"))
    ]  # Biến này chứa path tất cả ảnh từ image_folder

    for i, path in enumerate(tqdm(image_paths, desc="Processing images")):
        ocr_text = perform_ocr(
            path
        )  # trả về 1 câu tiếng Việt scan được từ ảnh. Vd: ocr_text:str = "dasdsdasdasdasdasdasdasd"
        embedding = get_embedding(
            ocr_text
        )  # trả về 1 vector từ SentenceBERT: np.ndarray
        index.add(i, embedding)  # nhét nó vào index usearch
        global_index2imgpath[i] = path  # chủ yếu là đánh dấu global_index -> image_path

    return index


def search(query: str, index: Index, top_k: int = 20) -> List[Tuple[int, float]]:
    """
    Thực hiện tìm kiếm trên index USearch dựa trên một câu truy vấn.

    Args:
        query (str): Câu truy vấn tìm kiếm.
        index (Index): Đối tượng Index của USearch.
        top_k (int, optional): Số lượng kết quả trả về. Mặc định là 5.

    Returns:
        List[Tuple[int, float]]: Danh sách các tuple (id, khoảng cách) của các kết quả tìm kiếm.
    """
    query_embedding = get_embedding(query)
    results = index.search(query_embedding, top_k)
    return [(int(match.key), match.distance) for match in results]
    # ở đây return kiểu như thế này [(global_index_1, điểm float 0.99), (2, 0.85), ...]


def draw_bounding_boxes(
    image_path: str, results: List[Tuple[List[Tuple[int, int]], str, float]]
) -> None:
    """
    Vẽ các hộp giới hạn và hiển thị văn bản OCR trên hình ảnh.

    Args:
        image_path (str): Đường dẫn đến file hình ảnh.
        results (List[Tuple[List[Tuple[int, int]], str, float]]): Kết quả OCR từ EasyOCR.
    """
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(1, figsize=(12, 8))

    ax.imshow(image)

    for bbox, text, prob in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox

        rect = plt.Rectangle(
            top_left,
            bottom_right[0] - top_left[0],
            bottom_right[1] - top_left[1],
            linewidth=2,
            edgecolor="g",
            facecolor="none",
        )

        ax.add_patch(rect)
        ax.text(
            top_left[0],
            top_left[1] - 10,
            f"{text} ({prob:.2f})",
            bbox=dict(facecolor="white", alpha=0.8),
            fontsize=8,
            color="green",
        )

    plt.axis("off")
    plt.tight_layout()
    plt.show()


def visualize_search_results(
    query: str, results: List[Tuple[int, float]], num_cols: int = 3
) -> None:
    """
    Hiển thị kết quả tìm kiếm dưới dạng bảng hình ảnh với điểm số.

    Args:
        query (str): Câu truy vấn tìm kiếm.
        results (List[Tuple[int, float]]): Kết quả tìm kiếm từ hàm search.
        num_cols (int, optional): Số cột trong bảng hiển thị. Mặc định là 3.
    """
    num_images = len(results)
    num_rows = (num_images + num_cols - 1) // num_cols

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(5 * num_cols, 5 * num_rows))
    fig.suptitle(f'Kết quả tìm kiếm cho: "{query}"', fontsize=16)

    for i, (id, distance) in enumerate(results):
        row = i // num_cols
        col = i % num_cols

        ax = axes[row, col] if num_rows > 1 else axes[col]

        image_path = global_index2imgpath[id]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax.imshow(image)
        ax.axis("off")
        ax.set_title(f"ID: {id}\nĐộ tương đồng: {1-distance:.4f}", fontsize=10)

    # Ẩn các trục thừa
    for i in range(num_images, num_rows * num_cols):
        row = i // num_cols
        col = i % num_cols
        ax = axes[row, col] if num_rows > 1 else axes[col]
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# Khởi tạo EasyOCR
reader = easyocr.Reader(["vi"])  # được sử dụng trong hàm perform_ocr

# Khởi tạo SentenceBERT, được sử dụng ở hàm get_embedding, và hàm get_embedding được sử dụng trong hàm process_image_and_build_index
model = SentenceTransformer("keepitreal/vietnamese-sbert")

# Biên toàn cục để lưu trữ ảnh ánh xạ index và đường dẫn hình ảnh
global_index2imgpath: Dict[int, str] = {}

In [ ]:
# index ở đây là cái Index của usearch.index, chứa các chỉ mục của từng vector
image_folder = "images/1/19"
index = process_image_and_build_index(image_folder)

In [ ]:
query = "người việt nam sẽ ủng hộ lệnh cấm nạn buôn bán thịt chó mèo"
results = search(query, index)  # ở đây nó trả về két quả,
# có cấu trúc dữ liệu là  [(global_index_1, điểm float 0.99), (2, 0.85), ...]